In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

from tqdm import tqdm

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id']

### csvをpickleにする

In [21]:
is_pickle = True
is_pickle = False

def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')

if is_pickle:
    new.sort_values(by=[key, 'purchase_date'], inplace=True)
    utils.to_df_pkl(path='../input', fname=f"new_merchant_transactions", df=new)



0it [00:00, ?it/s]

shape: (1963031, 14)




1it [00:00,  1.23it/s]

2it [00:01,  1.25it/s]

3it [00:02,  1.26it/s]



### Data Load

In [84]:
train = elo_load_data('train')
test = elo_load_data('test')
train_id = train[key].values
test_id = test[key].values
train_test = pd.concat([train, test], axis=0).reset_index(drop=True)
new = elo_load_data('new_')
# historical = elo_load_data('historical')




  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 118.03it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 191.41it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


 33%|███▎      | 1/3 [00:00<00:00,  3.71it/s]


 67%|██████▋   | 2/3 [00:00<00:00,  3.77it/s]


100%|██████████| 3/3 [00:00<00:00,  3.79it/s]




In [85]:
display(train.head())
print(train[key].drop_duplicates().shape)
print(train.shape)
display(new.head())
print(new.shape)
print(new[key].drop_duplicates().shape)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


(201917,)
(201917, 6)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1793298,Y,C_ID_00007093c1,76,N,1.0,B,222,M_ID_08f01305af,2,-0.671775,2018-04-03 11:13:35,3,2,21
1793299,Y,C_ID_00007093c1,69,N,1.0,B,879,M_ID_00a6ca8a8a,2,-0.656749,2018-04-09 16:23:59,1,9,29
38630,Y,C_ID_0001238066,314,N,1.0,B,307,M_ID_7d8102bb34,1,-0.732783,2018-03-01 16:48:27,1,9,19
38618,Y,C_ID_0001238066,314,N,1.0,B,367,M_ID_235e546dcc,1,-0.672136,2018-03-03 22:44:57,1,9,16
38633,Y,C_ID_0001238066,333,N,1.0,B,783,M_ID_a88790a464,1,-0.641722,2018-03-04 13:05:16,1,9,19


(1963031, 14)
(290001,)


#### New Transactionsの最少粒度は下記

In [118]:
print(new.shape)
cols = [key, 'purchase_date', 'purchase_amount', 'merchant_id', 'merchant_category_id', 'installments']
print(new[cols].drop_duplicates().shape)

(1963031, 14)
(1963031, 6)


SystemExit: 

### New Transactionsは一度の購買で数商品買っていると同じ時間で複数レコードが入る。そこの集計方法によりfeatureのあり方が変わる

### 直近購買日とのdiffと、そられの比率を特徴にする.

In [130]:
from joblib import Parallel, delayed
from dateutil.parser import parse

# ========================================================================
# Time Aggregate Function
# ========================================================================
# def diff_of_days(day1, day2):
def diff_of_days(args):
    uid = args[0]
    day1 = args[1]
    day2 = args[2]
    if str(type(day1)).count('int') or str(type(day2)).count('int'):
        return [day1, day2]
    days = (parse(day1) - parse(day2)).days
    return [uid, days, day1]

# 前回購買日との差分
def diff_shift_date(df, lag):
    
    # shiftは正が下方向、負が上方向にズレる
    df = df[[key, 'purchase_date']].drop_duplicates()
    df['shift1_date'] = df.groupby(key)['purchase_date'].shift(lag)
    df = df[[key, 'purchase_date', 'shift1_date']].dropna()
    id_list = df[key].values
    shift1 = df['shift1_date'].values
    shift0 = df['purchase_date'].values
    p_list = Parallel(n_jobs=-1)([delayed(diff_of_days)(args) for args in zip(id_list, shift0, shift1)])
    
    df_diff = pd.DataFrame(np.array(p_list), columns=[key, f'diff_days_lag{lag}@', 'purchase_date'])
    return df_diff
    
for i in tqdm(range(1, 4, 1)):
    diff = diff_shift_date(new, i)
    new = new.merge(diff, how='left', on=[key, 'purchase_date']) 
    
for col in [col for col in new.columns if col.count('@')]:
    new[col] = new[col].astype('float')
    
# 比率を計算
new['ratio_days_lag1_2@'] = new['diff_days_lag1@'] / new['diff_days_lag2@']
new['ratio_days_lag1_3@'] = new['diff_days_lag1@'] / new['diff_days_lag3@']
new['ratio_days_lag2_3@'] = new['diff_days_lag2@'] / new['diff_days_lag3@']





  0%|          | 0/3 [00:00<?, ?it/s]



 33%|███▎      | 1/3 [00:59<01:59, 59.66s/it]



 67%|██████▋   | 2/3 [01:53<00:57, 57.86s/it]



100%|██████████| 3/3 [02:38<00:00, 54.10s/it]





In [ ]:
new = new.sort_values(by=[key, 'purchase_date'])
new = utils.row_number(df=new, level=key)

### New Transactionsで作成した, nunique feature/lag1, lag2 nunique fewture/date_diff, date_ratio featureをjoin

In [74]:
train = train_test.loc[train_id, :]
test = train_test.loc[test_id, :]
for col in train.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = train[col].values, path=f'../features/1_first_valid/train_{col}')
        utils.to_pkl_gzip(obj = test[col].values, path=f'../features/1_first_valid/test_{col}')

MemoryError: 